# Составить словарь из столбца датафрейма вида: "слово" : "сколько раз оно встречаеться"

## Текстовая информация в столбце  G31_1 (Наименование и характеристики товаров) файла "3208.xlsx"
### Нужно составить словарь, для того что бы понимать какое слово(желательно существительное) чаще других встречаеться. Для определения существительного выделял окончания

In [28]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem.snowball import SnowballStemmer

G31_1_description = 'G31_1 (Наименование и характеристики товаров)'
G011_direction = 'G011 (Направление перемещения)' #импорт - экспорт
G022_name_sender = 'G022 (Наименование отправителя)'
G080_recipient_OGRN = 'G080 (ОГРН получателя)'
G081_recipient_INN = 'G081 (ИНН получателя)'
G082_recipient_Name = 'G082 (Наименование получателя)'
G221_currency = 'G221 (Букв.код валюты контракта)'
G35_gross_weight = 'G35 (Вес брутто, кг)'
G38_net_weight = 'G38 (Вес нетто, кг)'
G12_customs_value_total = 'G12 (Общая таможенная стоимость по ГТД)'
G222_invoice_value_total = 'G222 (Общая фактурная стоимость по ГТД)'
G23_exchange_rates = 'G23 (Курс валюты)'
G42_invoice_value = 'G42 (Фактурная стоимость)'
G42RUB_invoice_value_rub = 'G42RUB (Фактурная стоимость в рублях)'
G45_customs_value = 'G45 (Таможенная стоимость)'
G46_statistic_value = 'G46 (Статистическая стоимость, USD.)'
usd_kg = 'USDKG (USD за КГ)'
G30_city_warehouse = 'G30CITY (Город склада)'

#нужные нам стольбцы
columns = [G31_1_description, G022_name_sender, G080_recipient_OGRN, G081_recipient_INN, 
           G082_recipient_Name, G221_currency, G23_exchange_rates, 
           G30_city_warehouse, G38_net_weight, G42_invoice_value, G42RUB_invoice_value_rub, G46_statistic_value, usd_kg]

#функция токенизации и стемминга
def token_and_stem(text):
    tokens  = [word for word in nltk.word_tokenize(text)] # выделяем слова
    #print(tokens, '\n')
    filt_tokens = []
    for token in tokens:
        if re.search('[а-яА-Яa-zA-Z]', token): #убираем цифры
            if len(token) > 3: #смотрим, что бы слова состояли минимум из 4 символов
                filt_tokens.append(token)
    #print(filt_tokens, '\n')
    stems = [] #стемминг
    for token in filt_tokens:
        if re.search('[а-яА-Я]', token): #для русских слов стемминг
            stems.append(stemmer_rus.stem(token))
        elif re.search('[a-zA-Z]', token):#для английских слов стемминг
            stems.append(stemmer_eng.stem(token))
    #print(stems)
    return stems

df_excel = pd.read_excel('3208.xlsx')#считываем данные в датафрейм

mask = df_excel[G011_direction] == 'ИМ'#выбираем направление ИМпорт
df = df_excel.copy()[mask]
df = df[columns]
df = df.groupby(G31_1_description).aggregate(sum)#группируем данные
df = df.reset_index()
df[G31_1_description] = df[G31_1_description].str.lower() #нижний регистр
# начинаем отделять знаки препинания
df[G31_1_description] = df[G31_1_description].str.replace(',', ' ') 
df[G31_1_description] = df[G31_1_description].str.replace('.', ' ') 
df[G31_1_description] = df[G31_1_description].str.replace('-', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(';', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(':', ' ')
df[G31_1_description] = df[G31_1_description].str.replace('(', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(')', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(r'[\W]+', ' ')



In [29]:
# в тексте используеться английский и русский язык
stemmer_rus = SnowballStemmer('russian')
stemmer_eng = SnowballStemmer('english')

#создаем стоп слова из англ и русского алфавита
file_name_stopwords = 'stopwords-ru.txt'
f = open(file_name_stopwords, encoding='utf8').read()
stopwords_rus = f.split('\n')
stopwords_eng = nltk.corpus.stopwords.words('english')
stopwords = []
stopwords.extend(stopwords_rus)
stopwords.extend(stopwords_eng)


In [30]:
df.shape

(17424, 7)

In [31]:
df.head()

,G31_1 (Наименование и характеристики товаров),G23 (Курс валюты),"G38 (Вес нетто, кг)",G42 (Фактурная стоимость),G42RUB (Фактурная стоимость в рублях),"G46 (Статистическая стоимость, USD.)",USDKG (USD за КГ)
0,duro tak 87 4287 продукт на основе сополимера...,73.7247,180.00,429.97,31699,501.26,2.78
1,gel base avon wl15 раствор гелеобразователя б...,73.5145,0.99,62.35,4584,74.37,75.12
2,gel base avon wl15 раствор гелеобразователя б...,70.5618,0.50,31.49,2222,38.90,77.80
3,gel base avon wl15 раствор гелеобразователя б...,143.3280,1.50,94.47,6699,114.55,151.39
4,kleiberit праймер с содержанием дихлорметана ...,69.0243,11550.00,17556.00,1211791,22694.43,1.96


In [45]:
#Описание товаров загоняем в одну текстовую переменную для составления словаря вида: "слово" : "сколько раз встречаеться"
text = ''
for paragraph in df[G31_1_description]:
    text += paragraph
len(text)

16311162

In [49]:
text[:1000]

' duro tak 87 4287 продукт на основе сополимера акрилата с ацетатом не сод этил спирт используется как покрытие клей остаточной липкости для нанесения на трансдермальные пластыри 1 бочка 180 кг состав сополимер акрилата с ацетатом 30 50 этилацетат 59 винилацетат 2 henkel technologies belgium n v отсутствует duro tak 87 4287 1207594 отсутствует 180 gel base avon wl15 раствор гелеобразователя бутилен изопропилен этеновый сополимер 15 в летучем органическом растворителе изододекан 85 gel base avon wl15 раствор гелеобразователя бутилен изопропилен этеновый сополимер 15 в летучем органическом растворителе изододекан 85 в виде бесцветной жидкости нерастворимой в воде сырьё для парфюмерно косметической промышленности не для пищевых целей не содержит этилового спирта применяется в качестве загустителя плёнкообразователя arch chemicals inc отсутствует r21750 99 gel base avon wl15 раствор гелеобразователя бутилен изопропилен этеновый сополимер 15 в летучем органическом растворителе изододекан 85

In [104]:
#Составляем список слов. 
def tokenizerMy(text):
    tokens  = [word for word in nltk.word_tokenize(text)] # выделяем слова
    #print(tokens, '\n')
    filt_tokens = []
    
    #массивы окончания прилагательных
    adjective_endings_2 = ['ый', "ий", "ое", "ёе", 'ой', 'ая', 'ые', 'ие',
                           'ей', 'ых', 'их', 'ым', 'им', 'ую', 'ее', 'ом', 'ем']
    adjective_endings_3 = ['ого', 'его','ому', 'ему']
    
    for token in tokens:
        if (re.search('[а-яА-Я]', token)) and not(re.search('[0-9]', token)): #убираем цифры и слова которые содержат цифры
            if len(token) > 4: #смотрим, что бы слова состояли минимум из 5 символов
                if not((token[-2:] in adjective_endings_2) or (token[-3:] in adjective_endings_3)):#убираем прилагательные
                    #исключаем из списка так же слова с данными окончаниями
                    if (token[-1] != 'я'):
                        if (token[-1] != 'е'):
                            if (token[-2:] != 'ит'):
                                if (token[-2:] != 'ет'):
                                    if (token[-1] != 'и'):
                                        if (token[-2:] != 'ях'):
                                            if (token[-2:] != 'ах'):
                                                if (token[-2:] != 'ов'):
                                                    if (token[-1] != 'ы'):
                                                        filt_tokens.append(token)
    #print(filt_tokens, '\n')
    return filt_tokens

In [105]:
token_text = tokenizerMy(text)
len(token_text)

446332

In [106]:
token_text

['продукт',
 'сополимера',
 'акрилата',
 'спирт',
 'бочка',
 'состав',
 'сополимер',
 'акрилата',
 'этилацетат',
 'винилацетат',
 'раствор',
 'бутилен',
 'изопропилен',
 'сополимер',
 'изододекан',
 'раствор',
 'бутилен',
 'изопропилен',
 'сополимер',
 'изододекан',
 'сырьё',
 'парфюмерно',
 'спирта',
 'раствор',
 'бутилен',
 'изопропилен',
 'сополимер',
 'изододекан',
 'сырьё',
 'парфюмерно',
 'раствор',
 'бутилен',
 'изопропилен',
 'сополимер',
 'изододекан',
 'сырьё',
 'парфюмерно',
 'спирта',
 'упаковка',
 'ведро',
 'артикул',
 'праймер',
 'дихлорметана',
 'полимера',
 'перед',
 'праймер',
 'бочек',
 'класс',
 'производства',
 'грунт',
 'глицен',
 'сополимера',
 'раствор',
 'спирта',
 'ацетона',
 'аэроз',
 'антикорроз',
 'грунт',
 'глицен',
 'сополимера',
 'раствор',
 'спирта',
 'ацетона',
 'аэроз',
 'антикорроз',
 'метилэтилкетон',
 'бутанон',
 'растворитель',
 'глицен',
 'сополимер',
 'углерод',
 'бутилацетат',
 'эдиметиленфенил',
 'диизоцианата',
 'изоцианато',
 'триметил',
 'из

In [107]:
#составляем словарь по тексту с указанием сколько раз встречаеться данное словов в тексте
from collections import Counter
dict_text = Counter(token_text)

In [108]:
len(dict_text)

5814

In [109]:
print(dict_text)

Counter({'состав': 29757, 'смола': 17604, 'спирта': 11668, 'бутилацетат': 10074, 'ксилол': 9859, 'ацетат': 9183, 'краска': 8419, 'нафта': 7922, 'этилбензол': 7775, 'растворитель': 7028, 'эмаль': 6987, 'спирт': 5791, 'смесь': 5712, 'основа': 5582, 'диоксид': 5578, 'титана': 5229, 'работ': 4528, 'сольвент': 4441, 'этилацетат': 4315, 'грунт': 3819, 'ксилен': 3628, 'толуол': 3507, 'пигмент': 3396, 'ксилола': 3367, 'бутил': 3271, 'тальк': 3265, 'пропанол': 3021, 'отвердитель': 2854, 'содержат': 2736, 'метил': 2733, 'алкид': 2701, 'цинка': 2687, 'полимер': 2687, 'упаковка': 2492, 'красок': 2469, 'бутанол': 2435, 'ремонта': 2399, 'расфасован': 2251, 'аэрозоль': 2199, 'бутан': 2175, 'вещество': 2160, 'грунтовка': 2063, 'триметилбензол': 2030, 'ацетон': 2000, 'веществ': 1971, 'пропан': 1904, 'метилэтилкетон': 1882, 'брутто': 1882, 'метилэтил': 1849, 'расфасована': 1811, 'нефть': 1759, 'средств': 1731, 'метилэтилацетат': 1704, 'компонент': 1648, 'оксид': 1621, 'эфира': 1604, 'содерж': 1593, 'бан

In [110]:
#записываем в Series для записи в Excel
series_text = pd.Series(dict_text)

In [111]:
series_text.to_excel('Dict_LKM.xlsx')